### 主表日期和股票代码处理

In [ ]:
import pandas as pd
import numpy as np

def process_stock_repo_data(file_path, output_path="./data/interim/1_股票回购数据.xlsx"):
    """
    处理股票回购数据Excel文件，新增Year和Code列
    
    参数:
    file_path: 输入Excel文件路径
    output_path: 输出Excel文件路径（可选，默认在原文件名后加_processed）
    """
    # 读取Excel文件
    print("正在读取Excel文件...")
    try:
        # 读取两个sheet
        df_sheet1 = pd.read_excel(file_path, sheet_name='sheet1', header=1)
        df_sheet2 = pd.read_excel(file_path, sheet_name='sheet2', header=1)
    except Exception as e:
        print(f"读取文件失败: {e}")
        return
    
    print(f"原始数据行数: {len(df_sheet1)}")
    
    # 处理sheet1
    print("正在处理sheet1数据...")
    
    # 确保列名正确
    if '代码' not in df_sheet1.columns or '简称' not in df_sheet1.columns:
        print("错误: '代码'或'简称'列不存在")
        return
    
    # 1. 新增Code列（提取股票代码的数字部分）
    print("正在提取股票代码数字部分...")
    df_sheet1['Code'] = df_sheet1['代码'].astype(str).apply(
        lambda x: x.split('.')[0] if '.' in x else x
    )
    
    # 2. 新增Year列（从最新公告日期提取年份）
    print("正在提取最新公告日期的年份...")
    # 先确保日期列是datetime类型
    if '最新公告日期' in df_sheet1.columns:
        # 转换日期列
        df_sheet1['最新公告日期'] = pd.to_datetime(df_sheet1['最新公告日期'], errors='coerce')
        # 提取年份
        df_sheet1['Year'] = df_sheet1['最新公告日期'].dt.year
    else:
        print("警告: '最新公告日期'列不存在")
        df_sheet1['Year'] = np.nan
    
    # 3. 调整列顺序：在"代码"列后插入Code列，在"简称"列后插入Year列
    print("正在调整列顺序...")
    
    # 获取所有列的列表
    cols = list(df_sheet1.columns)
    
    # 移除新增的列
    cols.remove('Code')
    cols.remove('Year')
    
    # 找到"代码"和"简称"列的位置
    code_index = cols.index('代码') if '代码' in cols else -1
    name_index = cols.index('简称') if '简称' in cols else -1
    
    # 在适当位置插入新列
    if code_index >= 0:
        cols.insert(code_index + 1, 'Code')
    else:
        cols.insert(1, 'Code')  # 如果找不到，插入到第二列
    
    if name_index >= 0:
        # 注意：Code列插入后，索引可能会变化
        adjusted_name_index = cols.index('简称')
        cols.insert(adjusted_name_index + 1, 'Year')
    else:
        # 如果找不到"简称"列，尝试找其他列
        if '最新公告日期' in cols:
            date_index = cols.index('最新公告日期')
            cols.insert(date_index, 'Year')
        else:
            cols.append('Year')
    
    # 按新顺序排列列
    df_sheet1 = df_sheet1[cols]
    
    # 输出处理结果摘要
    print("\n=== 处理结果摘要 ===")
    print(f"处理后的列数: {len(df_sheet1.columns)}")
    print(f"处理后的行数: {len(df_sheet1)}")
    print(f"前5行Year列的值: {df_sheet1['Year'].head().tolist()}")
    print(f"前5行Code列的值: {df_sheet1['Code'].head().tolist()}")
    
    
    print(f"\n正在保存到: {output_path}")
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        df_sheet1.to_excel(writer, sheet_name='sheet1', index=False)
        df_sheet2.to_excel(writer, sheet_name='sheet2', index=False)
    
    print(f"处理完成! 文件已保存到: {output_path}")
    
    # 显示处理后数据的前几行
    print("\n=== 处理后数据预览 ===")
    print(df_sheet1[['代码', 'Code', '简称', 'Year', '最新公告日期']].head(10))
    
    return df_sheet1, df_sheet2

# 使用示例
if __name__ == "__main__":
    # 指定你的Excel文件路径
    input_file = "./data/raw/股票回购明细20251206.xlsx"  # 修改为你的文件路径
    
    # 处理数据
    processed_data = process_stock_repo_data(input_file)

正在读取Excel文件...
原始数据行数: 14487
正在处理sheet1数据...
正在提取股票代码数字部分...
正在提取最新公告日期的年份...
正在调整列顺序...

=== 处理结果摘要 ===
处理后的列数: 30
处理后的行数: 14487
前5行Year列的值: [2022.0, 2016.0, 2023.0, 2022.0, 2021.0]
前5行Code列的值: ['000002', '000002', '000004', '000004', '000004']

正在保存到: ./data/interim/1_股票回购数据.xlsx
处理完成! 文件已保存到: ./data/interim/1_股票回购数据.xlsx

=== 处理后数据预览 ===
          代码    Code     简称    Year     最新公告日期
0  000002.SZ  000002    万科A  2022.0 2022-07-02
1  000002.SZ  000002    万科A  2016.0 2016-01-15
2  000004.SZ  000004  *ST国华  2023.0 2023-07-27
3  000004.SZ  000004  *ST国华  2022.0 2022-07-08
4  000004.SZ  000004  *ST国华  2021.0 2021-07-29
5  000008.SZ  000008   神州高铁  2023.0 2023-01-17
6  000008.SZ  000008   神州高铁  2019.0 2019-01-10
7  000008.SZ  000008   神州高铁  2019.0 2019-01-10
8  000008.SZ  000008   神州高铁  2019.0 2019-01-10
9  000008.SZ  000008   神州高铁  2019.0 2019-01-10


In [5]:
import pandas as pd
import numpy as np

def process_stock_repo_data(file_path, output_path="./data/interim/1_股票回购数据.xlsx"):
    """
    处理股票回购数据Excel文件，新增Year和Code列
    
    参数:
    file_path: 输入Excel文件路径
    output_path: 输出Excel文件路径（可选，默认在原文件名后加_processed）
    """
    # 读取Excel文件
    print("正在读取Excel文件...")
    try:
        # 读取两个sheet，指定使用第二行作为表头
        df_sheet1 = pd.read_excel(file_path, sheet_name='sheet1', header=1)
        df_sheet2 = pd.read_excel(file_path, sheet_name='sheet2', header=1)
    except Exception as e:
        print(f"读取文件失败: {e}")
        return
    
    print(f"原始数据行数: {len(df_sheet1)}")
    
    # 处理sheet1
    print("正在处理sheet1数据...")
    
    # 确保列名正确
    if '代码' not in df_sheet1.columns or '简称' not in df_sheet1.columns:
        print("错误: '代码'或'简称'列不存在")
        return
    
    # 1. 新增Code列（提取股票代码的数字部分）
    print("正在提取股票代码数字部分...")
    df_sheet1['Code'] = df_sheet1['代码'].astype(str).apply(
        lambda x: x.split('.')[0] if '.' in x else x
    )
    
    # 2. 新增Year列（从最新公告日期提取年份）
    print("正在提取最新公告日期的年份...")
    # 先确保日期列是datetime类型
    if '最新公告日期' in df_sheet1.columns:
        # 转换日期列
        df_sheet1['最新公告日期'] = pd.to_datetime(df_sheet1['最新公告日期'], errors='coerce')
        # 提取年份
        df_sheet1['Year'] = df_sheet1['最新公告日期'].dt.year
        
        # 将日期列格式化为只显示日期（不显示时间）
        df_sheet1['最新公告日期'] = df_sheet1['最新公告日期'].dt.date
    else:
        print("警告: '最新公告日期'列不存在")
        df_sheet1['Year'] = np.nan
    
    # 3. 处理其他日期列，移除时间部分
    print("正在处理其他日期列...")
    
    # 定义需要处理的日期列
    date_columns = ['首次公布完成日期', '预案日', '最新公告日期']
    
    for col in date_columns:
        if col in df_sheet1.columns:
            try:
                # 确保是datetime类型
                if not pd.api.types.is_datetime64_any_dtype(df_sheet1[col]):
                    df_sheet1[col] = pd.to_datetime(df_sheet1[col], errors='coerce')
                
                # 转换为日期格式（移除时间部分）
                df_sheet1[col] = df_sheet1[col].dt.date
            except Exception as e:
                print(f"警告: 处理列 '{col}' 时出错: {e}")
    
    # 4. 调整列顺序：在"代码"列后插入Code列，在"简称"列后插入Year列
    print("正在调整列顺序...")
    
    # 获取所有列的列表
    cols = list(df_sheet1.columns)
    
    # 移除新增的列
    cols.remove('Code')
    cols.remove('Year')
    
    # 找到"代码"和"简称"列的位置
    code_index = cols.index('代码') if '代码' in cols else -1
    name_index = cols.index('简称') if '简称' in cols else -1
    
    # 在适当位置插入新列
    if code_index >= 0:
        cols.insert(code_index + 1, 'Code')
    else:
        cols.insert(1, 'Code')  # 如果找不到，插入到第二列
    
    if name_index >= 0:
        # 注意：Code列插入后，索引可能会变化
        adjusted_name_index = cols.index('简称')
        cols.insert(adjusted_name_index + 1, 'Year')
    else:
        # 如果找不到"简称"列，尝试找其他列
        if '最新公告日期' in cols:
            date_index = cols.index('最新公告日期')
            cols.insert(date_index, 'Year')
        else:
            cols.append('Year')
    
    # 按新顺序排列列
    df_sheet1 = df_sheet1[cols]
    
    # 输出处理结果摘要
    print("\n=== 处理结果摘要 ===")
    print(f"处理后的列数: {len(df_sheet1.columns)}")
    print(f"处理后的行数: {len(df_sheet1)}")
    print(f"前5行Year列的值: {df_sheet1['Year'].head().tolist()}")
    print(f"前5行Code列的值: {df_sheet1['Code'].head().tolist()}")
    
    # 显示日期处理结果
    print("\n=== 日期列处理结果示例 ===")
    date_cols_to_show = [col for col in date_columns if col in df_sheet1.columns]
    if date_cols_to_show:
        print(df_sheet1[date_cols_to_show].head(5))
    
    # 保存到新文件
    if output_path is None:
        # 默认在原文件名后加_processed
        import os
        base_name = os.path.splitext(file_path)[0]
        output_path = f"{base_name}_processed.xlsx"
    
    print(f"\n正在保存到: {output_path}")
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        df_sheet1.to_excel(writer, sheet_name='sheet1', index=False)
        df_sheet2.to_excel(writer, sheet_name='sheet2', index=False)
    
    print(f"处理完成! 文件已保存到: {output_path}")
    
    # 显示处理后数据的前几行
    print("\n=== 处理后数据预览 ===")
    preview_cols = ['代码', 'Code', '简称', 'Year']
    date_preview_cols = [col for col in date_columns if col in df_sheet1.columns]
    preview_cols.extend(date_preview_cols)
    print(df_sheet1[preview_cols].head(10))
    
    return df_sheet1, df_sheet2

# 使用示例
if __name__ == "__main__":
    # 指定你的Excel文件路径
    input_file = "./data/raw/股票回购明细20251206.xlsx"  # 修改为你的文件路径
    
    # 处理数据
    processed_data = process_stock_repo_data(input_file)

正在读取Excel文件...
原始数据行数: 14487
正在处理sheet1数据...
正在提取股票代码数字部分...
正在提取最新公告日期的年份...
正在处理其他日期列...
正在调整列顺序...

=== 处理结果摘要 ===
处理后的列数: 30
处理后的行数: 14487
前5行Year列的值: [2022.0, 2016.0, 2023.0, 2022.0, 2021.0]
前5行Code列的值: ['000002', '000002', '000004', '000004', '000004']

=== 日期列处理结果示例 ===
     首次公布完成日期         预案日      最新公告日期
0  2022-07-02  2022-03-31  2022-07-02
1  2016-01-15  2015-08-26  2016-01-15
2         NaT  2023-04-29  2023-07-27
3  2022-07-08  2022-04-30  2022-07-08
4  2021-07-29  2021-04-28  2021-07-29

正在保存到: ./data/interim/1_股票回购数据.xlsx
处理完成! 文件已保存到: ./data/interim/1_股票回购数据.xlsx

=== 处理后数据预览 ===
          代码    Code     简称    Year    首次公布完成日期         预案日      最新公告日期
0  000002.SZ  000002    万科A  2022.0  2022-07-02  2022-03-31  2022-07-02
1  000002.SZ  000002    万科A  2016.0  2016-01-15  2015-08-26  2016-01-15
2  000004.SZ  000004  *ST国华  2023.0         NaT  2023-04-29  2023-07-27
3  000004.SZ  000004  *ST国华  2022.0  2022-07-08  2022-04-30  2022-07-08
4  000004.SZ  000004  *ST国华  2021.0  

### 副表合并

In [9]:
import pandas as pd
import numpy as np
import os
import sys

def merge_stock_repo_with_dividend(repo_file, dividend_file, output_file):
    """
    合并股票回购表和分红实施表
    
    参数:
    repo_file: 股票回购表文件路径
    dividend_file: 分红实施表文件路径
    output_file: 输出文件路径
    """
    print(f"开始合并数据...")
    print(f"股票回购表: {repo_file}")
    print(f"分红实施表: {dividend_file}")
    
    # 1. 读取股票回购表
    print("1/5 正在读取股票回购表...")
    try:
        # 尝试使用header=1读取（跳过第一行标题）
        repo_df = pd.read_excel(repo_file, sheet_name='sheet1')
        print(f"读取股票回购表成功，行数: {len(repo_df)}")
        print(f"回购表列名: {list(repo_df.columns)}")
    except Exception as e:
        print(f"读取股票回购表失败: {e}")
        print("尝试使用默认方式读取...")
        repo_df = pd.read_excel(repo_file, sheet_name='sheet1')
        print(f"读取成功，行数: {len(repo_df)}")
    
    # 2. 读取分红实施表
    print("2/5 正在读取分红实施表...")
    try:
        # 分红实施表也有两行表头，第一行是英文，第二行是中文
        dividend_df = pd.read_excel(dividend_file, sheet_name='分红实施内容')
        print(f"读取分红实施表成功，行数: {len(dividend_df)}")
        print(f"分红实施表列名: {list(dividend_df.columns)}")
    except Exception as e:
        print(f"读取分红实施表失败: {e}")
        print("尝试使用默认方式读取...")
        dividend_df = pd.read_excel(dividend_file, sheet_name='分红实施内容')
        print(f"读取成功，行数: {len(dividend_df)}")
    
    # 3. 预处理数据
    print("3/5 正在预处理数据...")
    
    # 确保回购表中的Code列存在（纯数字股票代码）
    if 'Code' not in repo_df.columns and '代码' in repo_df.columns:
        print("在回购表中创建Code列...")
        repo_df['Code'] = repo_df['代码'].astype(str).apply(
            lambda x: x.split('.')[0] if '.' in x else x
        )
    
    # 确保回购表中的Year列是整数
    if 'Year' in repo_df.columns:
        repo_df['Year'] = pd.to_numeric(repo_df['Year'], errors='coerce').astype('Int64')
    
    # 处理股票回购表中的日期列（移除时间部分）
    print("处理股票回购表中的日期列...")
    repo_date_columns = ['最新公告日期', '首次公布完成日期', '预案日']
    
    for col in repo_date_columns:
        if col in repo_df.columns:
            try:
                # 转换为datetime并移除时间部分
                repo_df[col] = pd.to_datetime(repo_df[col], errors='coerce').dt.date
                print(f"  已处理列 '{col}'")
            except Exception as e:
                print(f"  警告: 处理列 '{col}' 时出错: {e}")
    
    # 处理分红表中的股票代码
    if 'SCode' in dividend_df.columns:
        # 提取纯数字股票代码
        dividend_df['Code'] = dividend_df['SCode'].astype(str).apply(
            lambda x: x.split('.')[0] if '.' in x else x
        )
        print(f"分红表Code列示例: {dividend_df['Code'].head(5).tolist()}")
    else:
        print("错误: 分红实施表中没有'SCode'列")
        print(f"可用列名: {list(dividend_df.columns)}")
        return
    
    # 处理分红表中的年份
    if 'Year' in dividend_df.columns:
        dividend_df['Year'] = pd.to_numeric(dividend_df['Year'], errors='coerce').astype('Int64')
        print(f"分红表Year列示例: {dividend_df['Year'].head(5).tolist()}")
    elif '会计年度' in dividend_df.columns:
        dividend_df['Year'] = pd.to_numeric(dividend_df['会计年度'], errors='coerce').astype('Int64')
        print(f"分红表Year列示例: {dividend_df['Year'].head(5).tolist()}")
    else:
        print("错误: 分红实施表中没有找到年份列")
        print(f"可用列名: {list(dividend_df.columns)}")
        return
    
    # 处理分红实施表中的日期列（移除时间部分）
    print("处理分红实施表中的日期列...")
    dividend_date_columns = ['股本基准日', '实施方案公告日', '股权登记日', '除权除息日', '派息日', '红股上市日']
    
    for col in dividend_date_columns:
        if col in dividend_df.columns:
            try:
                # 转换为datetime并移除时间部分
                dividend_df[col] = pd.to_datetime(dividend_df[col], errors='coerce').dt.date
                print(f"  已处理列 '{col}'")
            except Exception as e:
                print(f"  警告: 处理列 '{col}' 时出错: {e}")
    
    # 4. 合并数据
    print("4/5 正在合并数据...")
    
    # 显示合并前的行数
    print(f"合并前回购表行数: {len(repo_df)}")
    print(f"合并前分红表行数: {len(dividend_df)}")
    
    # 选择要合并的分红表列
    dividend_cols_to_merge = ['Code', 'Year']
    
    # 添加分红表的所有列（除了已经存在的Code和Year）
    for col in dividend_df.columns:
        if col not in ['Code', 'Year', 'SCode', '会计年度']:
            dividend_cols_to_merge.append(col)
    
    print(f"将从分红表合并的列: {[col for col in dividend_cols_to_merge if col not in ['Code', 'Year']]}")
    
    # 合并数据
    try:
        # 左连接：保留回购表的所有行，匹配分红表的对应行
        merged_df = pd.merge(
            repo_df,
            dividend_df[dividend_cols_to_merge],
            on=['Code', 'Year'],
            how='left',
            suffixes=('', '_分红')
        )
        
        print(f"合并后数据行数: {len(merged_df)}")
        print(f"合并后数据列数: {len(merged_df.columns)}")
        
        # 检查匹配情况
        if len(dividend_cols_to_merge) > 2:
            # 使用第一个非Code、Year的列来检查匹配情况
            check_col = dividend_cols_to_merge[2]
            matched_rows = merged_df[check_col].notna().sum()
            print(f"成功匹配的行数: {matched_rows} ({(matched_rows/len(merged_df)*100):.1f}%)")
        else:
            print("无法检查匹配情况：没有足够的分红列")
        
    except Exception as e:
        print(f"合并数据失败: {e}")
        print("尝试使用备用合并方法...")
        
        # 备用方法：逐个列合并
        merged_df = repo_df.copy()
        for col in dividend_cols_to_merge:
            if col not in ['Code', 'Year']:
                merged_df[col] = None
        
        # 尝试匹配每一行
        matched_count = 0
        for idx, row in merged_df.iterrows():
            code = row['Code']
            year = row['Year']
            
            # 在分红表中查找匹配项
            match = dividend_df[(dividend_df['Code'] == str(code)) & (dividend_df['Year'] == year)]
            
            if not match.empty:
                matched_count += 1
                # 填充匹配的分红数据
                for col in dividend_cols_to_merge:
                    if col not in ['Code', 'Year']:
                        merged_df.at[idx, col] = match.iloc[0][col]
        
        print(f"合并后数据行数: {len(merged_df)}")
        print(f"成功匹配的行数: {matched_count} ({(matched_count/len(merged_df)*100):.1f}%)")
    
    # 5. 保存结果
    print("5/5 正在保存文件...")
    
    # 确保输出目录存在
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 保存合并后的数据
    try:
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            merged_df.to_excel(writer, sheet_name='合并结果', index=False)
        
        print(f"合并完成! 文件已保存到: {output_file}")
        
        # 验证日期格式
        print("\n=== 日期列格式验证 ===")
        date_columns_to_check = ['最新公告日期', '首次公布完成日期', '预案日']
        for col in date_columns_to_check:
            if col in merged_df.columns:
                sample_value = merged_df[col].iloc[0] if len(merged_df) > 0 else None
                print(f"{col}: 示例值 = {sample_value}, 数据类型 = {type(sample_value)}")
        
    except Exception as e:
        print(f"保存文件失败: {e}")
        return
    
    # 显示合并结果示例
    print("\n=== 合并结果示例 ===")
    
    # 找出哪些列是来自分红表的
    dividend_columns = []
    for col in merged_df.columns:
        if col in dividend_df.columns and col not in repo_df.columns:
            dividend_columns.append(col)
    
    print(f"来自分红表的列: {dividend_columns[:5]}...")  # 只显示前5个
    
    if dividend_columns:
        # 显示前5行包含分红数据的行
        sample_cols = ['Code', 'Year', '简称'] if '简称' in merged_df.columns else ['Code', 'Year']
        sample_cols.extend(['最新公告日期', '首次公布完成日期', '预案日'])  # 添加日期列用于验证
        sample_cols.extend(dividend_columns[:3])  # 只显示前3个分红列
        
        # 筛选出有分红数据的行
        if len(dividend_columns) > 0:
            has_dividend = merged_df[dividend_columns[0]].notna()
            if has_dividend.any():
                print("有分红数据的行（前3行）:")
                print(merged_df[sample_cols][has_dividend].head(3).to_string())
            else:
                print("没有找到匹配的分红数据")
                print("前3行数据:")
                print(merged_df[sample_cols].head(3).to_string())
    
    return merged_df

# 使用示例
if __name__ == "__main__":
    # 获取命令行参数
    if len(sys.argv) >= 4:
        repo_file = sys.argv[1]
        dividend_file = sys.argv[2]
        output_file = sys.argv[3]
    else:
        # 如果未提供命令行参数，使用默认值或提示输入
        print("请提供文件路径参数:")
        print("使用方法: python merge_tables.py <回购表文件> <分红表文件> <输出文件>")
        print("示例: python merge_tables.py 股票回购明细20251206_processed.xlsx 分红实施内容.xlsx 合并结果.xlsx")
        
        # 或者使用默认值（根据你的实际情况修改）
        repo_file = "./data/interim/1_股票回购数据.xlsx"
        dividend_file = "./data/raw/分红实施内容.xlsx"
        output_file = "./data/interim/2_股票回购与分红合并.xlsx"
    
    # 执行合并
    merged_data = merge_stock_repo_with_dividend(repo_file, dividend_file, output_file)

请提供文件路径参数:
使用方法: python merge_tables.py <回购表文件> <分红表文件> <输出文件>
示例: python merge_tables.py 股票回购明细20251206_processed.xlsx 分红实施内容.xlsx 合并结果.xlsx
开始合并数据...
股票回购表: ./data/interim/1_股票回购数据.xlsx
分红实施表: ./data/raw/分红实施内容.xlsx
1/5 正在读取股票回购表...
读取股票回购表成功，行数: 14487
回购表列名: ['Unnamed: 0', '代码', 'Code', '简称', 'Year', '最新公告日期', '回购进度', '首次公布完成日期', '回购方式', '回购目的', '已回购数量(股)', '占总股本比例(%)', '已回购金额(元)', '回购均价', '占公告回购总资金比例(%)', '占公告回购总数量比例(%)', '较最新收盘价涨跌幅(%)', '公告回购数量(股）', '公告回购金额(万元）', '公告回购金额上限(万元)', '公告回购金额下限(万元)', '占总股本比例(%).1', '价格上限(元)', '与最新收盘价差异(元)', '股份类型', '资金来源', '预案日', '所属行业', '币种', '是否已注销']
2/5 正在读取分红实施表...
读取分红实施表成功，行数: 40605
分红实施表列名: ['SCode', 'CoName', 'Year', 'ImProgress', 'IDPR_PreTax', 'IDPR_AftTax', 'IBnsIssRt', 'IEqTnRt', 'EqBenmk', 'EqBenmkDt', 'AnnDt', 'RecordDt', 'DRDt', 'DPRDt', 'BnsShrLDt']
3/5 正在预处理数据...
处理股票回购表中的日期列...
  已处理列 '最新公告日期'
  已处理列 '首次公布完成日期'
  已处理列 '预案日'
分红表Code列示例: ['股票代码', '000001', '000001', '000001', '000001']
分红表Year列示例: [<NA>, 1991, 1992, 1993, 1994]
处理分红实施表中的

### 得到Treat值

In [12]:
import pandas as pd
import numpy as np

# 读取合并后的Excel文件
df = pd.read_excel('./data/interim/2_股票回购与分红合并.xlsx')

# 检查列名，确认是否有CoName列
print("数据框的列名：")
print(df.columns.tolist())

# 查看SCode_分红列的情况
if 'CoName' in df.columns:
    print(f"\nCoName列的情况：")
    print(f"非空值数量：{df['CoName'].notna().sum()}")
    print(f"空值数量：{df['CoName'].isna().sum()}")
    
    # 创建Treat列
    df['Treat'] = df['CoName'].apply(lambda x: 1 if pd.notna(x) else 0)
    
    # 或者更简洁的方式：
    # df['Treat'] = df['CoName'].notna().astype(int)
    
else:
    print("未找到CoName列，尝试查找其他可能的列名...")
    # 查找包含"分红"的列名
    div_cols = [col for col in df.columns if '分红' in str(col)]
    print(f"找到的包含'分红'的列：{div_cols}")
    
    if div_cols:
        # 使用第一个包含"分红"的列
        div_col = div_cols[0]
        print(f"使用列 '{div_col}' 来创建Treat列")
        df['Treat'] = df[div_col].apply(lambda x: 1 if pd.notna(x) else 0)
    else:
        # 如果找不到相关列，尝试找公司名称列
        if 'CoName' in df.columns:
            print("使用CoName列来创建Treat列")
            df['Treat'] = df['CoName'].apply(lambda x: 1 if pd.notna(x) else 0)
        else:
            # 查找是否有任何包含公司名称的列
            name_cols = [col for col in df.columns if 'Name' in str(col) or '公司' in str(col)]
            if name_cols:
                print(f"使用列 '{name_cols[0]}' 来创建Treat列")
                df['Treat'] = df[name_cols[0]].apply(lambda x: 1 if pd.notna(x) else 0)
            else:
                print("未找到合适的列来创建Treat列")

# 显示Treat列的统计信息
if 'Treat' in df.columns:
    print(f"\nTreat列统计：")
    print(f"Treat=1的行数：{df['Treat'].sum()}")
    print(f"Treat=0的行数：{(df['Treat']==0).sum()}")
    print(f"Treat=1的比例：{df['Treat'].mean():.2%}")
    


# 保存处理后的文件
output_file = './data/interim/3_股份回购与分红合并股份回购与分红合并_添加Treat.xlsx'
df.to_excel(output_file, index=False)
print(f"\n文件已保存为：{output_file}")

数据框的列名：
['Unnamed: 0', '代码', 'Code', '简称', 'Year', '最新公告日期', '回购进度', '首次公布完成日期', '回购方式', '回购目的', '已回购数量(股)', '占总股本比例(%)', '已回购金额(元)', '回购均价', '占公告回购总资金比例(%)', '占公告回购总数量比例(%)', '较最新收盘价涨跌幅(%)', '公告回购数量(股）', '公告回购金额(万元）', '公告回购金额上限(万元)', '公告回购金额下限(万元)', '占总股本比例(%).1', '价格上限(元)', '与最新收盘价差异(元)', '股份类型', '资金来源', '预案日', '所属行业', '币种', '是否已注销', 'CoName', 'ImProgress', 'IDPR_PreTax', 'IDPR_AftTax', 'IBnsIssRt', 'IEqTnRt', 'EqBenmk', 'EqBenmkDt', 'AnnDt', 'RecordDt', 'DRDt', 'DPRDt', 'BnsShrLDt']

CoName列的情况：
非空值数量：6408
空值数量：8079

Treat列统计：
Treat=1的行数：6408
Treat=0的行数：8079
Treat=1的比例：44.23%

文件已保存为：./data/interim/3_股份回购与分红合并股份回购与分红合并_添加Treat.xlsx


### 得到Post值

In [15]:
import pandas as pd
import numpy as np

# 读取已添加Treat列的文件
df = pd.read_excel('./data/interim/3_股份回购与分红合并股份回购与分红合并_添加Treat.xlsx')

print("数据基本信息：")
print(f"总行数：{len(df)}")
print(f"总列数：{len(df.columns)}")
print(f"列名：{df.columns.tolist()}")
print("-" * 50)

# 处理股票回购表中的日期列（移除时间部分）
print("处理股票回购表中的日期列...")
repo_date_columns = ['最新公告日期', '首次公布完成日期', '预案日']

for col in repo_date_columns:
    if col in df.columns:
        try:
            # 转换为datetime并移除时间部分
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.date
            print(f"  已处理列 '{col}'")
        except Exception as e:
            print(f"  警告: 处理列 '{col}' 时出错: {e}")

# 检查年份列的名称
# 可能是'Year_repo'、'Year'或其他名称
year_columns = [col for col in df.columns if 'Year' in str(col) or 'year' in str(col)]
print(f"找到的年份相关列：{year_columns}")

if year_columns:
    # 通常使用第一个年份列
    year_col = year_columns[0]
    print(f"使用列 '{year_col}' 来判断年份")
    
    # 确保年份列是数值类型
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    
    # 创建Post列：2023年及以前为0，之后为1
    df['Post'] = df[year_col].apply(lambda x: 0 if pd.notna(x) and x <= 2023 else 1 if pd.notna(x) else np.nan)
    
    # 统计Post列的值
    print("\nPost列统计：")
    print(f"Post=0 (≤2023年) 的行数：{(df['Post']==0).sum()}")
    print(f"Post=1 (>2023年) 的行数：{(df['Post']==1).sum()}")
    print(f"Post为空的行程：{df['Post'].isna().sum()}")
    
    # 显示年份范围
    print(f"\n年份范围：{df[year_col].min()} - {df[year_col].max()}")
    
    # 按年份和Post分组统计
    print("\n按年份和Post的分布：")
    year_post_count = df.groupby([year_col, 'Post']).size().unstack(fill_value=0)
    print(year_post_count.head(10))
    
else:
    print("未找到年份列！")
    print("可用的列名：")
    for col in df.columns:
        print(f"  {col}: 数据类型={df[col].dtype}, 示例={df[col].iloc[0] if len(df) > 0 else '空'}")
    
    # 尝试手动指定年份列
    # 这里假设可能叫'BPAmtDy'或其他名称
    possible_year_cols = ['BPAmtDy', 'Year', 'year', '年度', '会计年度']
    for col in possible_year_cols:
        if col in df.columns:
            year_col = col
            print(f"\n手动找到年份列: {year_col}")
            break
    else:
        # 如果没有找到，让用户输入
        year_col = input("请输入年份列的名称: ")
    
    # 确保年份列是数值类型
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    
    # 创建Post列
    df['Post'] = df[year_col].apply(lambda x: 0 if pd.notna(x) and x <= 2023 else 1 if pd.notna(x) else np.nan)

# 保存修改后的文件
output_file = './data/processed/股份回购与分红合并_添加Treat_Post.xlsx'
df.to_excel(output_file, index=False)
print(f"\n文件已保存为：{output_file}")

# 显示添加后的列
print(f"\n添加Post列后的列名：")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")

# 保存一份简明的统计报告
with open('Post列添加统计.txt', 'w', encoding='utf-8') as f:
    f.write("Post列添加统计报告\n")
    f.write("=" * 50 + "\n")
    f.write(f"总行数: {len(df)}\n")
    f.write(f"Post=0 (≤2023年): {(df['Post']==0).sum()} ({df['Post'].eq(0).mean():.2%})\n")
    f.write(f"Post=1 (>2023年): {(df['Post']==1).sum()} ({df['Post'].eq(1).mean():.2%})\n")
    f.write(f"Post为空值: {df['Post'].isna().sum()} ({df['Post'].isna().mean():.2%})\n")
    
    if year_columns:
        year_col = year_columns[0]
        # 按Post分组显示年份范围
        post_0_years = df[df['Post']==0][year_col].unique()
        post_1_years = df[df['Post']==1][year_col].unique()
        
        f.write(f"\nPost=0的年份: {sorted(post_0_years) if len(post_0_years) > 0 else '无'}\n")
        f.write(f"Post=1的年份: {sorted(post_1_years) if len(post_1_years) > 0 else '无'}\n")

print(f"\n统计报告已保存到：Post列添加统计.txt")

数据基本信息：
总行数：14487
总列数：44
列名：['Unnamed: 0', '代码', 'Code', '简称', 'Year', '最新公告日期', '回购进度', '首次公布完成日期', '回购方式', '回购目的', '已回购数量(股)', '占总股本比例(%)', '已回购金额(元)', '回购均价', '占公告回购总资金比例(%)', '占公告回购总数量比例(%)', '较最新收盘价涨跌幅(%)', '公告回购数量(股）', '公告回购金额(万元）', '公告回购金额上限(万元)', '公告回购金额下限(万元)', '占总股本比例(%).1', '价格上限(元)', '与最新收盘价差异(元)', '股份类型', '资金来源', '预案日', '所属行业', '币种', '是否已注销', 'CoName', 'ImProgress', 'IDPR_PreTax', 'IDPR_AftTax', 'IBnsIssRt', 'IEqTnRt', 'EqBenmk', 'EqBenmkDt', 'AnnDt', 'RecordDt', 'DRDt', 'DPRDt', 'BnsShrLDt', 'Treat']
--------------------------------------------------
处理股票回购表中的日期列...
  已处理列 '最新公告日期'
  已处理列 '首次公布完成日期'
  已处理列 '预案日'
找到的年份相关列：['Year']
使用列 'Year' 来判断年份

Post列统计：
Post=0 (≤2023年) 的行数：9141
Post=1 (>2023年) 的行数：5344
Post为空的行程：2

年份范围：2012.0 - 2025.0

按年份和Post的分布：
Post     0.0  1.0
Year             
2012.0    34    0
2013.0    92    0
2014.0   190    0
2015.0   240    0
2016.0   399    0
2017.0   521    0
2018.0   828    0
2019.0  1286    0
2020.0  1165    0
2021.0  1258    0

文件已保存为